In [422]:
import os
import networkx as nx
import numpy as np
import pandas as pd
import math
import pyflagser
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [423]:
G=nx.read_gml("Fully Observable/37busEx.gml")

In [424]:
A = nx.to_numpy_array(G)
print(A)

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [425]:
N=list(G.nodes)
print(N)
print(len(N))
E=list(G.edges)
print(E)
print(len(E))

['sourcebus', '799', '709', '775', '701', '702', '705', '713', '703', '727', '730', '704', '714', '720', '742', '712', '706', '725', '707', '724', '722', '708', '733', '732', '731', '710', '735', '736', '711', '741', '740', '718', '744', '734', '737', '738', '728', '729', '799r']
39
[('sourcebus', '799'), ('799', '799r'), ('709', '775'), ('709', '731'), ('709', '708'), ('709', '730'), ('701', '702'), ('701', '799r'), ('702', '705'), ('702', '713'), ('702', '703'), ('705', '742'), ('705', '712'), ('713', '704'), ('703', '727'), ('703', '730'), ('727', '744'), ('704', '714'), ('704', '720'), ('714', '718'), ('720', '707'), ('720', '706'), ('706', '725'), ('707', '724'), ('707', '722'), ('708', '733'), ('708', '732'), ('733', '734'), ('710', '735'), ('710', '736'), ('710', '734'), ('711', '741'), ('711', '740'), ('711', '738'), ('744', '728'), ('744', '729'), ('734', '737'), ('737', '738')]
38


In [426]:
N=list(G.nodes)
print(N)
print(len(N))
E=list(G.edges)
print(E)
print(len(E))

['sourcebus', '799', '709', '775', '701', '702', '705', '713', '703', '727', '730', '704', '714', '720', '742', '712', '706', '725', '707', '724', '722', '708', '733', '732', '731', '710', '735', '736', '711', '741', '740', '718', '744', '734', '737', '738', '728', '729', '799r']
39
[('sourcebus', '799'), ('799', '799r'), ('709', '775'), ('709', '731'), ('709', '708'), ('709', '730'), ('701', '702'), ('701', '799r'), ('702', '705'), ('702', '713'), ('702', '703'), ('705', '742'), ('705', '712'), ('713', '704'), ('703', '727'), ('703', '730'), ('727', '744'), ('704', '714'), ('704', '720'), ('714', '718'), ('720', '707'), ('720', '706'), ('706', '725'), ('707', '724'), ('707', '722'), ('708', '733'), ('708', '732'), ('733', '734'), ('710', '735'), ('710', '736'), ('710', '734'), ('711', '741'), ('711', '740'), ('711', '738'), ('744', '728'), ('744', '729'), ('734', '737'), ('737', '738')]
38


In [427]:
import pickle
with open('Fully Observable/fullanomalydataset.pkl', 'rb') as g:
    P0Data = pickle.load(g)

In [428]:
Class=[]
for i in range(500):
    Class.append(P0Data[i]["Anomalous"])
print(len(Class))
i=0
l=0
m=0
while i < len(Class):
    if Class[i] == 'No':
        l=l+1
        Class[i] = 0
    if Class[i] == 'Yes':
        m=m+1
        Class[i] = 1
    i += 1
#print(Class)
print(l)
print(m)

500
249
251


In [52]:
N_Senario=len(P0Data)
print(N_Senario)

500


In [429]:
def TimeSeries_Fe(TS_array, F_voltage):
    betti_0=[]
    for k in range(len(TS_array)):
        fec=[]
        AverageVoltage=[]
        Voltage=TS_array[k]
        for y in Voltage:
            AverageVoltage.append(Average(list(y)))
        #AverageVoltage = [i * 10 for i in AverageVoltage]
        for p in range(len(F_voltage)):
            n_active = np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                fec.append(0)
            else:
                b=A[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                fec.append(x[0])
            n_active.clear()
        betti_0.append(fec)
    return betti_0

In [430]:
TimeSeries_Voltage=P0Data[495]["TimeSeries_Voltage"]

In [431]:
#F_voltage=[105,104,103,102,100,99,98,97,96,95,94,93,92,90,88]
#F_voltage=[1.1,1.0,0.90,0.89,0.88]
F_voltage=[1.4,1.3,1.2,1.1,1.0,0.95,0.90,0.89,0.88,0.87,0.86,0.85,0.7]
betti=TimeSeries_Fe(TimeSeries_Voltage, F_voltage)

In [432]:
print(betti)

[[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1]]


In [238]:
from operator import add
for j in range
list( map(add, betti[0], betti[0]) )

[0, 2, 2, 2, 2]

In [438]:
F_voltage=[1.4,1.3,1.2,1.1,1.0,0.99,0.98,0.97,0.96,0.95,0.94,0.92,0.90,0.89,0.88,0.87,0.86,0.85]
Betti_0=[]
N_Senario=len(P0Data)
#N_Senario=5
for i in range(N_Senario):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(N_Senario-1)), end='', flush=True)
    TimeSeries_Voltage=P0Data[i]["TimeSeries_Voltage"]
    betti=TimeSeries_Fe(TimeSeries_Voltage, F_voltage)
    #print(betti)
    #Betti=Make_Sum(betti)
    #print(Betti)
    Betti_0.append(betti)
    #Betti_0.append(Betti)

Processing file 499 (100%)

In [396]:
X=Betti_0
y=Class
X=np.array(X)
y=np.array(y)

In [358]:
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [419]:
from sklearn.model_selection import KFold
# Define the K-fold Cross Validator
num_folds=12
kfold = KFold(n_splits=num_folds, shuffle=True)

In [440]:
X = Betti_0.copy()
y = Class.copy()
X=np.array(X)
y=np.array(y)

In [441]:
num_samples = 500
num_timesteps = 24
num_features = len(F_voltage)
num_classes = 2

In [455]:
loss_per_fold=[]
acc_per_fold=[]
fold_no = 1
for train, test in kfold.split(X, y):
    # Define the LSTM model
    model = Sequential([
        LSTM(10, activation='tanh',recurrent_activation='sigmoid',dropout=0.0,input_shape=(num_timesteps, num_features)),
        Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X[train], y[train], epochs=50, batch_size=32)
    # Evaluate the model
    #loss, accuracy = model.evaluate(X[test], y[test])
    #print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
    scores = model.evaluate(X[test], y[test])
    
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

      # Increase fold number
    fold_no = fold_no + 1

Epoch 1/50


2024-04-16 01:26:22.678275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 01:26:22.862918: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 01:26:23.230513: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 2s 60ms/step - loss: 0.6957 - accuracy: 0.5089
Epoch 2/50
15/15 [==============================] - 0s 13ms/step - loss: 0.6995 - accuracy: 0.4578
Epoch 3/50
15/15 [==============================] - 0s 10ms/step - loss: 0.6933 - accuracy: 0.5156
Epoch 4/50
15/15 [==============================] - 0s 26ms/step - loss: 0.6936 - accuracy: 0.5022
Epoch 5/50
15/15 [==============================] - 0s 9ms/step - loss: 0.6908 - accuracy: 0.5511
Epoch 6/50
15/15 [==============================] - 0s 9ms/step - loss: 0.6919 - accuracy: 0.5289
Epoch 7/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6912 - accuracy: 0.5267
Epoch 8/50
15/15 [==============================] - 0s 9ms/step - loss: 0.6885 - accuracy: 0.5711
Epoch 9/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6855 - accuracy: 0.5933
Epoch 10/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6823 - accuracy: 0.5578
Epoch 11/50
15/15 [=======

2024-04-16 01:26:31.783933: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 01:26:31.840451: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 111ms/step - loss: 0.5450 - accuracy: 0.7400
Score for fold 1: loss of 0.5449749231338501; accuracy of 74.00000095367432%
Epoch 1/50


...

2/2 [==============================] - 1s 137ms/step - loss: 0.4556 - accuracy: 0.8400
Score for fold 9: loss of 0.45561501383781433; accuracy of 83.99999737739563%
Epoch 1/50


2024-04-16 01:27:44.203901: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 01:27:44.315052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 01:27:44.633501: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 2s 79ms/step - loss: 0.7090 - accuracy: 0.5467
Epoch 2/50
15/15 [==============================] - 0s 9ms/step - loss: 0.7116 - accuracy: 0.4911
Epoch 3/50
15/15 [==============================] - 0s 10ms/step - loss: 0.7016 - accuracy: 0.5000
Epoch 4/50
15/15 [==============================] - 0s 9ms/step - loss: 0.6991 - accuracy: 0.4978
Epoch 5/50
15/15 [==============================] - 0s 9ms/step - loss: 0.6948 - accuracy: 0.5089
Epoch 6/50
15/15 [==============================] - 0s 10ms/step - loss: 0.6950 - accuracy: 0.4867
Epoch 7/50
15/15 [==============================] - 0s 11ms/step - loss: 0.6955 - accuracy: 0.4889
Epoch 8/50
15/15 [==============================] - 0s 9ms/step - loss: 0.6941 - accuracy: 0.5378
Epoch 9/50
15/15 [==============================] - 0s 10ms/step - loss: 0.6936 - accuracy: 0.5133
Epoch 10/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6931 - accuracy: 0.5067
Epoch 11/50
15/15 [======

2024-04-16 01:27:52.627021: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 01:27:52.681035: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 104ms/step - loss: 0.6267 - accuracy: 0.6400
Score for fold 10: loss of 0.6267407536506653; accuracy of 63.999998569488525%


In [456]:
print(acc_per_fold)

[74.00000095367432, 89.99999761581421, 83.99999737739563, 83.99999737739563, 74.00000095367432, 77.99999713897705, 68.00000071525574, 74.00000095367432, 83.99999737739563, 63.999998569488525]


In [457]:
import statistics
mean_value = statistics.mean(acc_per_fold)
std_dev = statistics.stdev(acc_per_fold)
print("Mean using LSTM", mean_value)
print("Standard Deviation:", std_dev)

Mean using LSTM 77.39999890327454
Standard Deviation: 8.113088451974987


In [256]:
def Make_Sum(Array):
    one_dim_array = []

    # Iterate through each row of the 2D array
    for row in Array:
        # Iterate through each element of the row
        for i, element in enumerate(row):
            # If the one-dimensional array is not large enough, extend it
            if len(one_dim_array) <= i:
                one_dim_array.append(element)
            else:
                # Add the element to the corresponding index of the one-dimensional array
                one_dim_array[i] += element
    return one_dim_array

    

In [458]:
def TimeSeries_Fe_MP(TS_voltage, TS_branchFlow, F_voltage,F_Flow):
    betti_0=[]
    for k in range(len(TS_voltage)):
        fec=[]
        AverageVoltage=[]
        Voltage=TS_voltage[k]
        for y in Voltage:
            AverageVoltage.append(Average(list(y)))
        #AverageVoltage = [i * 100 for i in AverageVoltage]
        BranchFlow=[]
        Branch_Flow=TS_branchFlow[k]
        for j  in range(len(Branch_Flow)):
            BranchFlow.append(Branch_Flow[j][0])

        for p in range(len(F_voltage)):
            Active_node_v=np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
            for q in range(len(F_Flow)):
                #if AverageVoltage[p]> F_voltage[p] and BranchFlow[q]> F_Flow[q]:
                #n_active = np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
                n_active=Active_node_v.copy()
                #print(n_active)
                G = nx.DiGraph()
                G.add_nodes_from(n_active)
                indices = np.where(np.array(BranchFlow) > F_Flow[q])[0].tolist()
                for s in indices:
                    a=int(N.index(E[s][0]))
                    b=int(N.index(E[s][1]))
                    if a in n_active and b in n_active:
                        G.add_edge(a, b)
                    #n_active.append(int(N.index(E[s][0])))
                    #n_active.append(int(N.index(E[s][1])))
                #Active_node=np.unique(n_active)
                #print(G.edges())
                if (len(n_active)==0):
                    fec.append(0)
                else:
                    #b=A[Active_node,:][:,Active_node]
                    Adj = nx.to_numpy_array(G)
                    my_flag=pyflagser.flagser_unweighted(Adj, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                    x = my_flag["betti"]
                    fec.append(x[0])
                n_active.clear()
            Active_node_v.clear()
        betti_0.append(fec)
    return betti_0




In [460]:
F_Flow=[100,90,75,50,25,10,5,0]
F_voltage=[1.4,1.3,1.2,1.1,1.0,0.99,0.98,0.97,0.96,0.95,0.94,0.92,0.90,0.89,0.88,0.87,0.86,0.85]
TimeSeries_Voltage=P0Data[0]["TimeSeries_Voltage"]
TimeSeries_Branch_Flow=P0Data[0]["BranchFlow"]
betti=TimeSeries_Fe_MP(TimeSeries_Voltage,TimeSeries_Branch_Flow, F_voltage,F_Flow)

In [463]:
print(len(betti))

24


In [466]:
F_Flow=[100,90,75,50,25,10,5,0]
F_voltage=[1.4,1.3,1.2,1.1,1.0,0.99,0.98,0.97,0.96,0.95,0.94,0.92,0.90,0.89,0.88,0.87,0.86,0.85]
Betti_0=[]
#N_Senario=len(P0Data)
N_Senario=300
for i in range(N_Senario):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(N_Senario-1)), end='', flush=True)
    TimeSeries_Voltage=P0Data[i]["TimeSeries_Voltage"]
    TimeSeries_Branch_Flow=P0Data[i]["BranchFlow"]
    betti=TimeSeries_Fe_MP(TimeSeries_Voltage,TimeSeries_Branch_Flow, F_voltage,F_Flow)
    #print(betti)
    #Betti=Make_Sum(betti)
    #print(Betti)
    Betti_0.append(betti)
    #Betti_0.append(Betti)

Processing file 299 (100%)

In [479]:
print(Betti_0[4])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 35, 34, 31, 25, 17, 9, 1, 36, 36, 35, 32, 26, 18, 10, 2, 36, 36, 35, 32, 26, 18, 10, 2, 36, 36, 35, 32, 26, 18, 10, 2, 37, 37, 36, 33, 27, 19, 11, 1, 37, 37, 36, 33, 27, 19, 11, 1, 37, 37, 36, 33, 27, 19, 11, 1, 37, 37, 36, 33, 27, 19, 11, 1, 37, 37, 36, 33, 27, 19, 11, 1, 37, 37, 36, 33, 27, 19, 11, 1, 37, 37, 36, 33, 27, 19, 11, 1, 37, 37, 36, 33, 27, 19, 11, 1, 37, 37, 36, 33, 27, 19, 11, 1, 37, 37, 36, 33, 27, 19, 11, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 35, 35, 32, 26, 19, 9, 1, 36, 36, 36, 33, 27, 20, 10, 2, 36, 36, 36, 33, 27, 20, 10, 2, 37, 37, 37, 34, 28, 21, 11, 1, 37, 37, 37, 34, 28, 21, 11, 1, 37, 37, 37, 34, 28, 21, 11, 1, 37, 37, 37, 34, 28, 21, 11, 1, 37, 37, 37, 34, 28, 21, 11, 1, 37, 37, 37, 34, 28, 21, 11, 1, 37, 37, 37, 34, 28, 21, 11, 1, 37, 37, 37, 34, 28, 21, 11, 1, 37, 37, 37, 34, 28, 21, 11, 1, 

In [467]:
Class=[]
for i in range(300):
    Class.append(P0Data[i]["Anomalous"])
print(len(Class))
i=0
l=0
m=0
while i < len(Class):
    if Class[i] == 'No':
        l=l+1
        Class[i] = 0
    if Class[i] == 'Yes':
        m=m+1
        Class[i] = 1
    i += 1
#print(Class)
print(l)
print(m)

300
147
153


In [480]:
from sklearn.model_selection import KFold
# Define the K-fold Cross Validator
num_folds=10
kfold = KFold(n_splits=num_folds, shuffle=True)
X = Betti_0.copy()
y = Class.copy()
X=np.array(X)
y=np.array(y)

In [488]:
from sklearn.preprocessing import MinMaxScaler

num_samples = 300
num_timesteps = 24
num_features = len(F_voltage)*len(F_Flow)
num_classes = 2
loss_per_fold=[]
acc_per_fold=[]
fold_no = 1
for train, test in kfold.split(X, y):
    # Define the LSTM model
    model = Sequential([
        LSTM(10, activation='tanh',recurrent_activation='sigmoid',dropout=0.0,input_shape=(num_timesteps, num_features)),
        Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    X0=[]
    for i in range(300):
        
        scaler = MinMaxScaler()

# Fit scaler to data and transform data
        X0.append(scaler.fit_transform(X[i]))
    X0=np.array(X0)
    #print(X0[0])
    # Train the model
    model.fit(X0[train], y[train], epochs=50, batch_size=32)
    # Evaluate the model
    #loss, accuracy = model.evaluate(X[test], y[test])
    #print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
    scores = model.evaluate(X0[test], y[test])
    
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

      # Increase fold number
    fold_no = fold_no + 1

Epoch 1/50


2024-04-16 23:28:07.218541: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:28:07.336664: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:28:07.799453: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 3s 144ms/step - loss: 0.7141 - accuracy: 0.4333
Epoch 2/50
9/9 [==============================] - 0s 26ms/step - loss: 0.6939 - accuracy: 0.5000
Epoch 3/50
9/9 [==============================] - 0s 18ms/step - loss: 0.6844 - accuracy: 0.5630
Epoch 4/50
9/9 [==============================] - 0s 14ms/step - loss: 0.6742 - accuracy: 0.5556
Epoch 5/50
9/9 [==============================] - 0s 11ms/step - loss: 0.6561 - accuracy: 0.5963
Epoch 6/50
9/9 [==============================] - 0s 11ms/step - loss: 0.6296 - accuracy: 0.6778
Epoch 7/50
9/9 [==============================] - 0s 12ms/step - loss: 0.5848 - accuracy: 0.7185
Epoch 8/50
9/9 [==============================] - 0s 18ms/step - loss: 0.5288 - accuracy: 0.7741
Epoch 9/50
9/9 [==============================] - 0s 11ms/step - loss: 0.4839 - accuracy: 0.7667
Epoch 10/50
9/9 [==============================] - 0s 9ms/step - loss: 0.4525 - accuracy: 0.8259
Epoch 11/50
9/9 [=======================

2024-04-16 23:28:14.260590: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:28:14.300846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 411ms/step - loss: 0.2852 - accuracy: 0.8667
Score for fold 1: loss of 0.2851705551147461; accuracy of 86.66666746139526%
Epoch 1/50


...

1/1 [==============================] - 1s 542ms/step - loss: 0.3687 - accuracy: 0.8667
Score for fold 9: loss of 0.36868816614151; accuracy of 86.66666746139526%
Epoch 1/50


2024-04-16 23:29:16.798600: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:29:16.886184: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:29:17.344439: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 2s 122ms/step - loss: 0.7076 - accuracy: 0.4852
Epoch 2/50
9/9 [==============================] - 0s 37ms/step - loss: 0.6926 - accuracy: 0.5111
Epoch 3/50
9/9 [==============================] - 0s 26ms/step - loss: 0.6844 - accuracy: 0.5074
Epoch 4/50
9/9 [==============================] - 0s 37ms/step - loss: 0.6810 - accuracy: 0.6185
Epoch 5/50
9/9 [==============================] - 0s 13ms/step - loss: 0.6751 - accuracy: 0.6630
Epoch 6/50
9/9 [==============================] - 0s 14ms/step - loss: 0.6657 - accuracy: 0.6111
Epoch 7/50
9/9 [==============================] - 0s 11ms/step - loss: 0.6552 - accuracy: 0.5778
Epoch 8/50
9/9 [==============================] - 0s 12ms/step - loss: 0.6357 - accuracy: 0.7074
Epoch 9/50
9/9 [==============================] - 0s 9ms/step - loss: 0.6111 - accuracy: 0.7593
Epoch 10/50
9/9 [==============================] - 0s 8ms/step - loss: 0.5759 - accuracy: 0.7852
Epoch 11/50
9/9 [========================

2024-04-16 23:29:23.650113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-04-16 23:29:23.699878: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 565ms/step - loss: 0.2415 - accuracy: 0.9333
Score for fold 10: loss of 0.2415003925561905; accuracy of 93.33333373069763%


In [489]:
import statistics
mean_value = statistics.mean(acc_per_fold)
std_dev = statistics.stdev(acc_per_fold)
print("Mean using LSTM", mean_value)
print("Standard Deviation:", std_dev)

Mean using LSTM 88.66666615009308
Standard Deviation: 5.018483700357382
